In [ ]:
!pip install pandas


In [ ]:
import pandas as pd
import glob
import os

## Read file 

In [ ]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Trip_chain/trip_chain_dwell_time.csv'

# Read the CSV file into a DataFrame
trip_summary_df= pd.read_csv(file_path)

In [ ]:
trip_summary_df = trip_summary_df.drop(columns=['age'])


In [ ]:
# Define the file path
file_path = '/Users/aparnaj8/Box/Data (Matthew Rizzo)/DataAnalysis/FormattingQAChecks/REDCap_Formatting/REDCap_Format_Data/demographics.csv'

# Read the CSV file into a DataFrame
demo_summary_df= pd.read_csv(file_path)

In [ ]:
demo_summary_df = demo_summary_df.rename(columns={'year': 'year_mapped'})


In [ ]:
demo_summary_df.head()

In [ ]:
# Define the file path
file_path =  '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Speeding_Variable/high_speed_road_60mph_or_greater.csv'

# Read the CSV file into a DataFrame
high_speed_df= pd.read_csv(file_path)

In [ ]:
high_speed_df.head()

### Merge files

In [ ]:
final_merged_df = trip_summary_df.merge(
    demo_summary_df[['subj', 'year_mapped', 'education_years', 'race', 'gender', 'age']],  # Select required columns from demo_summary_df
    on=['subj', 'year_mapped'],  # Merge on 'subj' and 'year_mapped' from both DataFrames
    how='left'  # Retain all rows from trip_summary_df
)


In [ ]:
final_merged_df.head()

#### AM PM peak variable

In [ ]:
final_merged_df['time_start_utc'] = pd.to_datetime(final_merged_df['time_start_utc'], errors='coerce')
final_merged_df['time_end_utc'] = pd.to_datetime(final_merged_df['time_end_utc'], errors='coerce')

In [ ]:
final_merged_df['time_start_cst'] = pd.to_datetime(final_merged_df['time_start_cst'], errors='coerce')
final_merged_df['time_end_cst'] = pd.to_datetime(final_merged_df['time_end_cst'], errors='coerce')

In [ ]:
import pandas as pd

def classify_time_of_day(timestamp):
    hour = timestamp.hour
    if 7 <= hour <= 9:
        return 'AM Peak'
    elif 16 <= hour <= 18:
        return 'PM Peak'
    else:
        return 'None'

# Example usage with a pandas dataframe
final_merged_df['AM_PM_peak'] = final_merged_df['time_start_cst'].apply(lambda x: classify_time_of_day(pd.to_datetime(x)))


In [ ]:
final_merged_df.head()

### Nighttime Daytime trip

In [ ]:
import pandas as pd
from datetime import timedelta

# Define nighttime period (9 PM to 6 AM)
night_start = 21  # 9 PM
night_end = 6     # 6 AM

# Function to check if a specific time is nighttime
def is_nighttime(timestamp):
    hour = timestamp.hour
    return night_start <= hour or hour < night_end

# Function to calculate the percentage of the trip that is nighttime
def calculate_nighttime_percentage(row):
    # Convert start and end time to datetime
    start_time = pd.to_datetime(row['time_start_cst'])
    end_time = pd.to_datetime(row['time_end_cst'])

    # Calculate the total trip duration in minutes
    total_duration = (end_time - start_time).total_seconds() / 60  # in minutes

    # Initialize counters for nighttime minutes
    nighttime_minutes = 0

    # Iterate through each minute of the trip
    current_time = start_time
    while current_time <= end_time:
        if is_nighttime(current_time):
            nighttime_minutes += 1
        current_time += timedelta(minutes=1)

    # Calculate the percentage of time that is nighttime
    nighttime_percentage = (nighttime_minutes / total_duration) * 100
    return nighttime_percentage

# Apply the function to each row and assign "nighttime" or "daytime"
final_merged_df['night_trips'] = final_merged_df.apply(calculate_nighttime_percentage, axis=1)

# Create a new column 'day_or_night' based on the nighttime percentage
final_merged_df['day_or_night'] = final_merged_df['night_trips'].apply(
    lambda x: 'nighttime' if x >= 80 else 'daytime'
)



In [ ]:
final_merged_df.head()

In [ ]:
# Drop specified columns
columns_to_drop = ['home_distance', 'label', 'trip_chain', 'time_diff_minutes', 'night_trips']
final_merged_df = final_merged_df.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
final_merged_df.dtypes

## Correct within 15 and 25 miles frm home variable

In [ ]:
import pandas as pd

# Add the '15_miles_from_home' column
final_merged_df['15_miles_from_home'] = final_merged_df['distance_miles'].apply(lambda x: 'yes' if x <= 15 else 'no')

# Add the '25_miles_from_home' column
final_merged_df['25_miles_from_home'] = final_merged_df['distance_miles'].apply(lambda x: 'yes' if 15 < x <= 25 else 'no')

In [ ]:
# Save the filtered DataFrame to CSV
final_merged_df.to_csv('final_list_of_var_v1.csv', index=False)

## Attach High Speed Information

In [ ]:
# Merge final_merged_df and high_speed_df on 'subj' and 'drive'
final_high_speed= pd.merge(final_merged_df, high_speed_df, on=['subj', 'drive'], how='left')


In [ ]:
# Rename the column
final_high_speed.rename(columns={'percentage_at_60mph_or_greater': 'high_speed_drive'}, inplace=True)

# Replace non-null values with 'Yes'
final_high_speed['high_speed_drive'] = final_high_speed['high_speed_drive'].notnull().replace({True: 'Yes', False: None})



In [ ]:
final_high_speed.head()

## Hard Braking information

In [ ]:
# Define the file path
file_path = r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Hard_Braking/final_agg_hard_braking_3551.csv'

# Read the CSV file into a DataFrame
hard_brake_df = pd.read_csv(file_path)

In [ ]:
hard_brake_df.head()

In [ ]:
# Group by 'subj' and 'drive', and count the occurrences of 'agg_hard_brake' for each combination
hard_brake_counts = (
    hard_brake_df.groupby(['subj', 'drive'])
    .agg(count=('agg_hard_brake', 'count'))
    .reset_index()
)

# Store the result in a DataFrame for further merging
hard_brake_counts_df = pd.DataFrame(hard_brake_counts)

In [ ]:
hard_brake_counts_df.head()

In [ ]:
# Merge final_high_speed and hard_brake_counts_df based on 'subj' and 'drive'
merged_df = pd.merge(final_high_speed, hard_brake_counts_df, on=['subj', 'drive'], how='left')

In [ ]:
merged_df.head()

In [ ]:
# Save the filtered DataFrame to CSV
merged_df.to_csv('final_list_of_var_v2.csv', index=False)

## Probability of stopping

In [ ]:
# Define the file path
file_path =  r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/final_list_of_var_v2.csv'

# Read the CSV file into a DataFrame
list_of_var = pd.read_csv(file_path)

In [ ]:
# Define the file path
file_path =  r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Stopping_at_stop_sign/probability_of_stopping_final.csv'

# Read the CSV file into a DataFrame
stoppage_df = pd.read_csv(file_path)

In [ ]:
list_of_var.head()

In [ ]:
# Rename the 'count' column
list_of_var.rename(columns={'count': 'hard_brake_count'}, inplace=True)

In [ ]:
stoppage_df.head()

## Attach probability of stopping for every subjec

In [ ]:
# Merge based on 'subj' and 'week_number'
final_df = pd.merge(list_of_var, stoppage_df, on=['subj', 'week_number'], how='left')

In [ ]:
final_df.head()

In [ ]:
# Drop the specified columns from final_df
final_df = final_df.drop(columns=['total_stop_signs', 'stoppages_yes'])


In [ ]:
final_df.dtypes

In [ ]:
len(final_df)

In [ ]:
import pandas as pd

# Ensure time_start_cst and time_end_cst are in datetime format
final_df['time_start_cst'] = pd.to_datetime(final_df['time_start_cst'])
final_df['time_end_cst'] = pd.to_datetime(final_df['time_end_cst'])

# Calculate the duration in minutes
final_df['calculated_minutes'] = (final_df['time_end_cst'] - final_df['time_start_cst']).dt.total_seconds() / 60

# Update distance_minutes where it is NaN and distance_miles > 0
condition = (final_df['duration_minutes'].isna()) & (final_df['distance_miles'] > 0)
final_df.loc[condition, 'duration_minutes'] = final_df.loc[condition, 'calculated_minutes']

# Drop the temporary 'calculated_minutes' column if it's no longer needed
final_df.drop(columns=['calculated_minutes'], inplace=True)


In [ ]:
# Count total NaN values in the entire DataFrame
total_nan = final_df.isna().sum().sum()
print(f"Total NaN values in the DataFrame: {total_nan}")

# Count NaN values for each column
nan_per_column = final_df.isna().sum()
print("\nNaN values per column:")
print(nan_per_column)


In [ ]:
# Save the filtered DataFrame to CSV
final_df.to_csv('final_list_of_var_v3.csv', index=False)

## Final cleaning and data addition

#### remove nan from subj type

In [ ]:
# Define the file path
file_path = '/Users/aparn/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/final_list_of_var_v3.csv'

# Read the CSV file into a DataFrame
final_df = pd.read_csv(file_path)

In [ ]:
# Count of NaN values in 'subj_type' column
nan_count = final_df['subj_type'].isna().sum()

print(f"Number of NaN values in 'subj_type': {nan_count}")


In [ ]:
len(final_df)

In [ ]:
final_df['subj'].nunique()

In [ ]:
# Remove rows where 'subj_type' is NaN
final_df = final_df.dropna(subset=['subj_type'])


In [ ]:
final_df.head()

### remove 2024 data

In [ ]:
# Count the rows where 'year' is 2024
count_2024 = final_df[final_df['year'] == 2024].shape[0]

# Display the count
print(f"Count of rows where year == 2024: {count_2024}")


### attach acceleration infromation 

In [ ]:
# Define the file path
file_path =  r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/average_accel_x_RWRAD.csv'

# Read the CSV file into a DataFrame
avg_accel = pd.read_csv(file_path)

In [ ]:
avg_accel.head()

In [ ]:
final_df1 = pd.merge(final_df, avg_accel, on=['subj', 'drive'], how='left')

In [ ]:
final_df1.head()

In [ ]:
len(final_df1)

In [ ]:
final_df1.dtypes

## Extract final list of variables

In [ ]:
weekly_stats_df = pd.DataFrame()


In [ ]:
import pandas as pd
import numpy as np

# Function to calculate all the required weekly statistics
def calculate_weekly_stats(df):
    # Convert 'time_start_utc' and 'time_end_utc' to datetime format if not already
    df['time_start_utc'] = pd.to_datetime(df['time_start_utc'])
    df['time_end_utc'] = pd.to_datetime(df['time_end_utc'])
    df['time_start_cst'] = pd.to_datetime(df['time_start_cst'])
    df['time_end_cst'] = pd.to_datetime(df['time_end_cst'])
    
    # Initialize an empty dictionary to store calculated values
    weekly_stats = {}
    
    # Total number of miles driven in a week
    weekly_stats['total_miles'] = df['distance_miles'].sum()

    # Total number of trips in a week
    weekly_stats['total_trips'] = df.shape[0]

    # Total miles driven in a week divided by total number of trips in a week
    weekly_stats['avg_miles_per_trip'] = weekly_stats['total_miles'] / weekly_stats['total_trips']

    # Total driving minutes in a week
    weekly_stats['total_minutes'] = df['duration_minutes'].sum()

    # Total minutes of driving in a week divided by total number of trips in a week
    weekly_stats['avg_minutes_per_trip'] = weekly_stats['total_minutes'] / weekly_stats['total_trips']

    # Number of trips in week not classified as nighttime
    weekly_stats['non_nighttime_trips'] = df[df['day_or_night'] != 'nighttime'].shape[0]

    # Percent of trips in week not classified as nighttime
    weekly_stats['percent_non_nighttime'] = (weekly_stats['non_nighttime_trips'] / weekly_stats['total_trips']) * 100

    # Number of trips in week during 7–9 AM on weekdays
    weekly_stats['am_7_9_trips'] = df[(df['AM_PM_peak'] == 'AM Peak')].shape[0]
    
    # Percent of trips in week during 7–9 AM on weekdays
    weekly_stats['percent_am_7_9'] = (weekly_stats['am_7_9_trips'] / weekly_stats['total_trips']) * 100

    # Number of trips during which at least 80% of a trip was during nighttime in week
    weekly_stats['nighttime_80_percent'] = df[df['day_or_night'] == 'nighttime'].shape[0]

    # Percent of trips during which at least 80% of a trip was during nighttime in week
    weekly_stats['percent_nighttime_80'] = (weekly_stats['nighttime_80_percent'] / weekly_stats['total_trips']) * 100

    # Number of trips in week during 4–6 PM on weekdays
    weekly_stats['pm_4_6_trips'] = df[(df['AM_PM_peak'] == 'PM Peak')].shape[0]
    
    # Percent of trips in week during 4–6 PM on weekdays
    weekly_stats['percent_pm_4_6'] = (weekly_stats['pm_4_6_trips'] / weekly_stats['total_trips']) * 100

    # Number of trips traveled in week within 15 miles of home
    weekly_stats['within_15_miles'] = df[df['15_miles_from_home'] == 'yes'].shape[0]

    # Percent of trips traveled in week within 15 miles of home
    weekly_stats['percent_within_15_miles'] = (weekly_stats['within_15_miles'] / weekly_stats['total_trips']) * 100

    # Number of trips traveled in week within 25 miles of home
    weekly_stats['within_25_miles'] = df[df['25_miles_from_home'] == 'yes'].shape[0]

    # Percent of trips traveled in week within 25 miles of home
    weekly_stats['percent_within_25_miles'] = (weekly_stats['within_25_miles'] / weekly_stats['total_trips']) * 100

    # Number of trip chains in week (chain is a series of trips marked by unique values in 'trip_chain_check')
    weekly_stats['trip_chains'] = df['trip_chain_check'].nunique()
    
    # Calculate the total miles driven for each trip chain
    total_miles_chains = df.groupby('trip_chain_check')['distance_miles'].sum()
    
    # Calculate the total driving minutes for each trip chain
    total_minutes_chains = df.groupby('trip_chain_check')['duration_minutes'].sum()
    
    # Calculate the total number of unique trip chains
    total_trip_chains = df['trip_chain_check'].nunique()
    
    # Calculate the total miles of chains divided by the number of trip chains
    avg_miles_per_chain = total_miles_chains.sum() / total_trip_chains if total_trip_chains != 0 else np.nan
    
    # Calculate the total driving minutes for chains divided by the number of trip chains
    avg_minutes_per_chain = total_minutes_chains.sum() / total_trip_chains if total_trip_chains != 0 else np.nan
    
    # Add these to the weekly stats dictionary
    weekly_stats['avg_miles_per_chain'] = avg_miles_per_chain
    weekly_stats['avg_minutes_per_chain'] = avg_minutes_per_chain

    # Make sure we only count 'Yes' and avoid NaN values
    weekly_stats['speed_60_mph'] = df[df['high_speed_drive'] == 'Yes'].shape[0]
    
    # Percentage of trips in week where 20% of distance traveled was at a high-speed road
    # If the total trips are non-zero, calculate percentage, else return NaN
    weekly_stats['percent_speed_60_mph'] = (weekly_stats['speed_60_mph'] / weekly_stats['total_trips']) * 100 if weekly_stats['total_trips'] != 0 else np.nan

    # The average speed of trips
    weekly_stats['avg_trip_speed'] = df['speed_mph_mean'].mean()

    # The average acceleration of trips
    weekly_stats['avg_trip_accel_x'] = df['average_accel_x'].mean()
    
    # Number of events with a deceleration rate <= -0.35 g in a week
    weekly_stats['hard_brake_events'] = df[df['hard_brake_count'] > 0].shape[0]

    # Handling stoppage_probability: keeping unique values per subj
    stoppage_values = df['stoppage_probability'].dropna().unique()
    if len(stoppage_values) > 0:
        weekly_stats['stoppage_probability'] = stoppage_values[0]  # Attach first unique value for subj

    return weekly_stats

# Group by subj, week_number, and year, then apply the calculation function
weekly_stats_df = final_df1.groupby(['subj', 'week_number', 'year']).apply(calculate_weekly_stats).apply(pd.Series)

# Reset index to ensure 'subj', 'week_number', and 'year' are in the columns
weekly_stats_df.reset_index(inplace=True)

# Fill NaNs in 'stoppage_probability' column with empty strings where no value exists
weekly_stats_df['stoppage_probability'] = weekly_stats_df['stoppage_probability'].fillna('')

# Ensure 'year' information is included in the final DataFrame

# Reorder columns to have 'subj', 'week_number', and 'year' as the first three columns
cols = ['subj', 'week_number', 'year'] + [col for col in weekly_stats_df.columns if col not in ['subj', 'week_number', 'year']]
weekly_stats_df = weekly_stats_df[cols]


# Reorder columns to have 'subj' and 'week_number' as the first two columns
#cols = ['subj', 'week_number'] + [col for col in weekly_stats_df.columns if col not in ['subj', 'week_number']]
#weekly_stats_df = weekly_stats_df[cols]

In [ ]:
weekly_stats_df.head()

In [ ]:
# Group by 'subj' to get unique years for each subject
multi_year_subj = final_df1.groupby('subj')['year'].nunique()

# Filter subjects that have more than one unique year
multi_year_subj = multi_year_subj[multi_year_subj > 1]

# Display the subjects with more than one unique year
print(multi_year_subj)


In [ ]:
# Filter the data for the subjects with more than one unique year
filtered_data = final_df1[final_df1['subj'].isin(multi_year_subj.index)]

# Group by 'subj' and check if 'subj_type' is consistent across years
subj_type_changes = filtered_data.groupby('subj').agg({
    'subj_type': 'nunique'
}).reset_index()

# Filter subjects where 'subj_type' is changing (i.e., having more than 1 unique 'subj_type')
changing_subj = subj_type_changes[subj_type_changes['subj_type'] > 1]

# Display subjects where 'subj_type' is changing
print(changing_subj)


In [ ]:
# Create a subset of final_df1 with additional columns
additional_columns = ['education_years', 'race', 'gender', 'age', 'subj_type']
final_df_subset = final_df1[['subj', 'year'] + additional_columns].drop_duplicates(subset=['subj', 'year'])

# Merge the dataframes based on 'subj' and 'year'
weekly_stats_df = weekly_stats_df.merge(final_df_subset, on=['subj', 'year'], how='left')



In [ ]:
weekly_stats_df.head()

In [ ]:
len(weekly_stats_df)

In [ ]:
# Save the final weekly statistics dataframe to a CSV file
weekly_stats_df.to_csv('weekly_stats_with_demo.csv', index=False)